In [ ]:
import cv2
import numpy as np
import tensorflow as tf

In [ ]:
# Eğitimli çöp sınıflandırma modelinizi yükleyin
model_yolu = "garbagemodel.h5"  # Model yolunuzu değiştirin
model = tf.keras.models.load_model(model_yolu)
BOYUT = 64

In [ ]:
# Bir görüntüyü model girişi için ön işleme fonksiyonu
def goruntuyu_on_isle(goruntu):
    # Görüntüyü modelinizin giriş boyutuna yeniden boyutlandırın
    goruntu = cv2.resize(goruntu, (BOYUT, BOYUT))
    # Piksel değerlerini normalleştirin (modelinizin 0 ile 1 arasındaki değerleri beklediğini varsayarak)
    goruntu = goruntu / 255.0
    # Toplu işleme için görüntü boyutunu genişletin (isteğe bağlı)
    goruntu = np.expand_dims(goruntu, axis=0)
    return goruntu

In [ ]:
kamera = cv2.VideoCapture(0)  # Doğru kamera indeksini girin

while True:
    # Kameradan bir kare yakalayın
    ret, frame = kamera.read()
    if not ret:
        break

    # Çerçeve görüntüsünü ön işleyin
    on_islemeli_goruntu = goruntuyu_on_isle(frame)

    # Modelinizi kullanarak tahmin yapın
    tahmin = model.predict(on_islemeli_goruntu)
    # En yüksek olasılığa sahip tahmin edilen sınıf dizinini alın
    tahmin_edilen_sınıf_dizini = np.argmax(tahmin)

    # Dizin bazında sınıf etiketini alın
    tahmin_edilen_etiket = kategoriler[tahmin_edilen_sınıf_dizini]
    # Tahmin olasılığını alın
    tahmin_olasılığı = tahmin[0][tahmin_edilen_sınıf_dizini] * 100  # Yüzdeye dönüştürün

    # Tahmin edilen nesnenin etrafına bir dikdörtgen çizin
    if tahmin_olasılığı > 0.5:  # Tahmin olasılığı 0.5'ten yüksekse
        # Dikdörtgeni çizin
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 255, 0), 2)

    # Tahmini çerçevede görüntüleyin
    cv2.putText(frame, f"{tahmin_edilen_etiket}: {tahmin_olasılığı:.2f}%", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Çerçeveyi görüntüleyin
    cv2.imshow('Çöp Sınıflandırma', frame)

    # 'q' tuşuna basıldığında çıkın
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


In [ ]:
# Kamera kaynaklarını serbest bırakın
kamera.release()
cv2.destroyAllWindows()